In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup
import json
from os import listdir
from os.path import isfile, join
from bs4 import Comment
import time
import os
import re
import shutil

## General Function

In [2]:
def last_scraping_website(website, full_data):
    "Nilai dari parameter website adalah bisnis.com dan detik finance"
    website_data = full_data[full_data.website == website]
    last_scraping = (website_data.published_at.max() + pd.DateOffset(days=1)).strftime("%Y-%m-%d")
    yesterday = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    return last_scraping, yesterday

def generate_date_range(start, end):
    """Generate tanggal dengan rentang tertentu. Parameter
    'Start' dan 'End' harus berformat Y-m-d. Misalnya 2001-01-27
    """
    
    start = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = datetime.datetime.strptime(end, "%Y-%m-%d") + datetime.timedelta(days=1)
    return [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

## Function to Scraping Bisnis.com

### Scraping Title

In [3]:
def bisnis_scrape_title_page(halaman, tanggal, category):
    """Melakukan request kehalaman list berita dari webiste bisnis.com 
    pada tanggal tertentu dengan halaman tertentu pada kategori tertentu"""
    
    URL = "https://www.bisnis.com/index/page/?c={}&d={}&per_page={}".format(category, tanggal, halaman)
    page = requests.get(URL, headers={'user-agent': 'My app'})
    return BeautifulSoup(page.content, 'html.parser')

def bisnis_extract_title_information(berita):
    """Mengambil informasi judul berita, link, dan waktu publish"""
    
    div = berita.find('div', class_="col-sm-8")
    a = div.select("h2 a")[0]
    return {
        "title" : a['title'],
        "category": div.select(".wrapper-description a")[0]['title'],
        "published_at" : div.select(".date")[0].text.strip(),
        "link": a['href']
    }

def bisnis_scrape_title_tanggal(tanggal, category):
    """Scraping judul pada tanggal tertentu"""
    
    halaman_sekarang = 1
    beritas = []

    # Dalam satu hari bisa terdiri dari banyak berita
    # sehingga bisa terdiri dari banyak halaman
    while True:
        soup = bisnis_scrape_title_page(halaman_sekarang, tanggal, category)
        list_berita = soup.find("ul", class_="list-news").find_all("li")
        
        # Jika list berita kosong maka sudah sampai dihalaman terakhir
        if (len(list_berita) == 1) and (list_berita[0].find("h2").text.strip() == "Tidak ada berita"):
            break
            
        for berita in list_berita:
            beritas.append(bisnis_extract_title_information(berita))

        halaman_sekarang += 1
        
    return beritas


def bisnis_scrape_title(start, end, category):
    """Scraping untuk rentang tertentu. Hasilnya akan disimpan pada folder data
    masih dalam bentuk json karena akan dilakukan penyimpanan setiap 150 hari
    """
    
    generated_date = generate_date_range(start, end)
    hasil = []
    status = []
    website = f"bisnis.com, {'Market' if category == 194 else 'Financial'}"
    print(f"Start Scraping {website} from {start} until {end}")
    
    iterate = 1
    for date in generated_date:
        tanggal_ymd = date.strftime("%Y-%m-%d")
        
        print(f"Start Scraping title in {website} category on {tanggal_ymd}")
        jumlah_berita = 0
        
        try:
            list_berita = bisnis_scrape_title_tanggal(tanggal_ymd, category)            
            status_scrape = "success"
            jumlah_berita = len(list_berita)
            
            hasil.append({ "date" : tanggal_ymd, "article_count": jumlah_berita, "data" : list_berita})
            
            status.append({ "date" : tanggal_ymd, "status" : status_scrape, "article_count": jumlah_berita,
                           "website" : website, "message" : None})
            
        except Exception as e:
            status_scrape = "failed"
            status.append({ "date" : tanggal_ymd, "status" : status_scrape, "article_count": None,
                            "website" : website, "message" : str(e)})
        
        print(f"Scraping {tanggal_ymd} {status_scrape}, numbers of article is {jumlah_berita}\n")
        
    hasil = pd.DataFrame(hasil)
    
    # memisahkan kolom data menjadi kolom-kolom terpisah
    data_full = None
    for index, row in hasil.iterrows():
        data_satu_hari = pd.DataFrame(row['data'])
        data_satu_hari['date'] = row['date']

        if data_full is None: data_full = pd.DataFrame(data_satu_hari)
        else: data_full = pd.concat([data_full, pd.DataFrame(data_satu_hari)])

    data_full['published_at'] = pd.to_datetime(data_full.published_at, format="%d %b %Y | %H:%M WIB")
    return status, data_full.sort_values("published_at").reset_index(drop=True)

### Scraping Content

In [4]:
def clean_artikel_bisnis(text, stop_words):
    """ Untuk melakukan preprocessing sederhana hasil scraping
    seperti menghapus line break, whitespace dan kata-kata yang
    tidak bermakna yang sering muncul diawal atau akhir artikel
    """
    if text is None:
        return text
    
    for sentance in stop_words:
        text = text.replace(sentance, "")

    # hapus whitespace diawal dan akhir artikel dan hapus semua line break
    text = text.strip().replace('\n', ' ').replace('\r', '')
    
    # hapus strip diawalan artikel
    text =  re.sub("^(-+)*?", "", text).strip()
    
    # jika awalannya seperti ini, hapus aja datanya, ga ada informasinya
    if text[:30] == "Simak berita lainnya seputar t":
        return None
    
    return text

def scraping_one_article_bisnis(data_title, stop_words):
    """Scraping satu halaman tertentu dengan menggunakan
    data title yang sudah di scraping sebelumnya
    """
    iterasi = 0
    status_error = []
    data_per_tanggal = []

    for tanggal, df, in data_title.groupby("date"):
        print(f"Scraping content article of bisnis.com on {tanggal}")

        for index, row in df.iterrows():
            tmp = {'title' : row['title'], 'link' : row['link'], 'date' : tanggal}

            try:
                page = requests.get(tmp['link'], headers={'user-agent': 'My app'})
                page = BeautifulSoup(page.content, 'html.parser')
                
                # Di bisnis.com ada beberapa artikel yang tidak dapat diakses secara
                # umum dan harus berbayar
                if len(page.find_all("div", class_="wrapper-premium-content")) > 0:
                        raise Exception("premium content")
                        
                # 2 paragraf terakhir dibuang karena hanya mengandung iklan
                article_body = page.select(".description .sticky-wrapper .col-sm-10")[0]
                paragraphs = article_body.find_all("p")[:-2]
                
                # setiap paragraf dibelakangnya akan ditambahkan titik dan spasi
                isi_artikel = " ".join([paragraph.text.strip().rstrip('.') + '. ' for paragraph in paragraphs])    
                tmp['content'] = clean_artikel_bisnis(isi_artikel, stop_words)
                data_per_tanggal.append(tmp)

            except Exception as error:
                status_error.append({'date' : tanggal, "website" : "bisnis.com",
                                     'link': tmp['link'], 'message' : str(error)})
                time.sleep(2)

            iterasi += 1
            # setiap 200 iterasi, istirahat 15 detik.
            if iterasi % 200 == 0:
                time.sleep(15)
                
    # jika status errornya kosong maka buat dataframe kosong
    column_names = ["date", "website", "link", "message"]
    status_error = pd.DataFrame(status_error) if len(status_error) > 0 else pd.DataFrame(columns = column_names)
    return status_error, pd.DataFrame(data_per_tanggal)

## Function to Scraping Detik Finance

### Scraping Title

In [5]:
def detik_scrape_title_page(halaman, tanggal):
    """Melakukan request kehalaman list judul detik finance 
    pada tanggal tertentu dengan halaman tertentu"""
    
    URL = "https://finance.detik.com/indeks/{}?date={}".format(halaman, tanggal)
    page = requests.get(URL)
    return BeautifulSoup(page.content, 'html.parser')

def detik_extract_title_information(berita):
    """Mengambil informasi judul berita, link, dan waktu publish"""
    
    media_text = berita.find("div", class_="media__text")

    media_title = media_text.find("h3", class_="media__title").find("a")
    title = media_title.text.strip()
    link = media_title["href"]
    kategori = link.split("/")[3].replace("-", " ")

    media_date = media_text.find("div", class_="media__date").find("span")["d-time"]

    return {"title" : title, "category" : kategori, "published_at" : media_date,"link" : link}

def detik_scrape_title_tanggal(tanggal):
    """Scraping judul artikel pada tanggal tertentu
    """
    
    is_first = True
    halaman_sekarang = 1
    jumlah_halaman = 100
    beritas = []

    # Dalam satu tanggal bisa saja beritanya banyak sehingga dipecah menjadi
    # beberapa halaman (pagination)
    while halaman_sekarang <= jumlah_halaman:
        soup = detik_scrape_title_page(halaman_sekarang, tanggal)

        list_berita = soup.find_all("article", class_="list-content__item")    
        if len(list_berita) > 0:
            for berita in list_berita:
                beritas.append(detik_extract_title_information(berita))

            # Jika halaman pertama, cek ada berapa halaman dengan
            # melihat pada jumlah pagination dibagian bawah halaman
            # kemudian update jumlah halaman sehingga loop 'while' bisa berakhir
            if is_first:
                is_first = False
                list_pagination = soup.find_all("a", class_="pagination__item")
                idx_last_page = len(list_pagination) - 2
                jumlah_halaman = int(list_pagination[idx_last_page].text)
        elif is_first:
            jumlah_halaman = 0

        halaman_sekarang += 1
        
    return beritas

def detik_scrape_title(start, end):
    """Scraping judul artikel untuk rentang tertentu.
    """
    
    generated_date = generate_date_range(start, end)
    hasil = []
    status = []    
    iterate = 1
    print(f"Start Scraping detik finance from {start} until {end}")
    
    for date in generated_date:
        tanggal_ymd = date.strftime("%Y-%m-%d")
        tanggal_mdy = date.strftime("%m/%d/%Y") # format url di detik finance m-d-y
        
        print(f"Start Scraping title in detik finance website on {tanggal_ymd}")
        jumlah_berita = 0
        try:
            list_berita= detik_scrape_title_tanggal(tanggal_mdy)
            status_scrape = "success"
            jumlah_berita = len(list_berita)

            hasil.append({ "date" : tanggal_ymd, "article_count": jumlah_berita, "data" : list_berita})
            status.append({ "date" : tanggal_ymd, "status" : status_scrape, "article_count": jumlah_berita,
                           "website" : "detik finance", "message" : None})
            
        except Exception as e:  
            status_scrape = "failed"
            status.append({ "date" : tanggal_ymd, "status" : status_scrape, "jumlah_berita": None, 
                           "website" : "detik finance", "message" : str(e)})

        print(f"Scraping {tanggal_ymd} {status_scrape}, numbers of article is {jumlah_berita}\n")
        
    hasil = pd.DataFrame(hasil)
    
    # pisahkan kolom data menjadi beberapa kolom
    data_full = None
    for index, row in hasil.iterrows():
        data_satu_hari = pd.DataFrame(row['data'])
        data_satu_hari['date'] = row['date']

        if data_full is None: data_full = pd.DataFrame(data_satu_hari)
        else: data_full = pd.concat([data_full, pd.DataFrame(data_satu_hari)])

    data_full = data_full.reset_index(drop=True)
    # data waktu yang di scraping merupakan waktu dalam bentuk timestamp sehingga perlu
    # dikonversi terlebih dahulu dan ditambah 7 jam agar mengikuti waktu WIB
    data_full["published_at"] = pd.to_datetime(data_full['published_at'], unit='s') + pd.DateOffset(hours=7)

    return pd.DataFrame(status), data_full

### Scraping Content

In [6]:
def clean_artikel_detik(text):
    """ Untuk melakukan preprocessing sederhana hasil scraping
    seperti menghapus line break, whitespace dan kata-kata yang
    tidak bermakna yang sering muncul diawal atau akhir artikel
    """
    
    if text is None:
        return text
    
    # hapus line break
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ').strip()
    
    # identifikasi tempat penulisan artikel dan hapus tulisannya
    tmp = text.split("-")
    if (len(tmp) > 1) and (len(tmp[0]) < 27):
        awalan = tmp[0] + "-"
        text = text.replace(awalan, "").strip()
        
    # Menghapus kata-kata yang tidak perlu
    sentances = [
        "Lanjut ke halaman berikutnya >>>",
        "berikut ini:",
        "Berikut daftar lengkapnya",
        "[Gambas:Video 20detik]",
        "http://bit.ly/18anniversary.",
        "Berikut informasi selengkapnya.",
        "redaksi@detikFinance.com.",
        "Berikut informasi selengkapnya:",
        "http://bit.ly/promoakhirpekan.",
        "Semoga bermanfaat"
    ]
    
    for sentance in sentances:
        text = text.replace(sentance, "")
    
    # List kata-kata yang sering muncul diakhir artikel
    text = re.sub('(Lihat juga Video: .+)$', "", text)
    text = re.sub('(Simak Video .+)$', "", text)
    text = re.sub('(Disclaimer: .+)$', "", text)
    text = re.sub('(Mau tahu informasi selengkapnya\? .+)$', "", text)
    text = re.sub('(Berikut berita selengkapnya .+)$', "", text)
    text = re.sub('(Untuk mengetahui informasi lebih lengkap, .+)$', "", text)
    text = re.sub('(Untuk informasi lebih lengkap, .+)$', "", text)
    text = re.sub('(Khusus Untuk 100 Orang Pembaca .+)$', "", text)
    text = re.sub('(Mau Menjadi Kaya Atau Bahkan Lebih Kaya .+)$', "", text)
    text = re.sub('(Untuk mengetahui promo lain yang berlangsung .+)$', "", text)
    text = re.sub('(Untuk promo lebih lengkap, .+)$', "", text)
    text = re.sub('(Untuk melihat promo lainnya, .+)$', "", text)
    text = re.sub('(Untuk mengetahui promo lainnya, .+)$', "", text)
    text = re.sub('(Untuk mengetahui promo .+)$', "", text)
    text = re.sub('(Dapatkan eBook Tentang "61 Rahasia .+)$', "", text)
    text = re.sub('(Sudah Siapkah Anda di Tahun 2018\? .+)$', "", text)
    text = re.sub('(Sudah Siapkah Anda di Tahun 2018 ini\? .+)$', "", text)
    text = re.sub('(Sudah Siapkah Anda Ditahun 2018\? .+)$', "", text)
    text = re.sub('(Saksikan juga video .+)$', "", text)
    text = re.sub('(Lihat Video: .+)$', "", text)
    text = re.sub('(Baca 5 berita .+)$', "", text)
    text = re.sub('(Bisa kirim ceritanya ke .+)$', "", text)
    text = re.sub('(Tonton video .+)$', "", text)
    text = re.sub('(Tonton juga Video: .+)$', "", text)    
    text = re.sub('(Untuk mengetahui informasi dari .+)$', "", text)
    text = re.sub('(Hasil Intisari Pemikiran Dan Praktek Saya .+)$', "", text)
    text = re.sub('(Dapatkan eBook Tentang .+)$', "", text)
    text = re.sub('(Mau Ikut Pelatihan dengan Materi Terbaru .+)$', "", text)
    text = re.sub('(Mau ikut diskusi soal Aturan Baru Taksi .+)$', "", text)
    text = re.sub('(Bagaimana Kaya Secara Aman di Tahun 2017 ke 2018 ini\? .+)$', "", text)
    text = re.sub('(Untuk mengetahui info lengkap .+)$', "", text)
    text = re.sub('(Mau eBook Tentang .+)$', "", text)
    text = re.sub('(Mau Menjadi Kaya Atau Bahkan Lebih Kaya\? .+)$', "", text)
    text = re.sub('(Ikuti terus berita tentang .+)$', "", text)
    text = re.sub('(Untuk melihat promo .+)$', "", text)
    text = re.sub('(GRATIS\. .+)$', "", text)
    text = re.sub('(Tonton juga .+)$', "", text)
    text = re.sub('(Hasil Intisari Pemikiran Dan Praktik Saya .+)$', "", text)
    text = re.sub('(Simak berita selengkapnya .+)$', "", text)
    text = re.sub('(Untuk mengetahui info .+)$', "", text)
    text = re.sub("(Siap 'Bantu Jokowi Cari Menteri'\? .+)$", "", text)
    text = re.sub("(bagi detikers yang .+)$", "", text)
    text = re.sub("(Ingin tahu kisah lain .+)$", "", text)
    text = re.sub("(Untuk ilmu yang lebih lengkap lagi, .+)$", "", text)
    text = re.sub("(Untuk mengetahui beragam promo .+)$", "", text)
    text = re.sub("(Simak ulasan lengkapnya .+)$", "", text)
    text = re.sub("(Di Jakarta dibuka workshop sehari tentang bagaimana .+)$", "", text)
    text = re.sub("(detikcom bersama BRI .+)$", "", text)
    text = re.sub("(detikcom bersama Bank BRI .+)$", "", text)
    text = re.sub("(Promo ini masih ditambah .+)$", "", text)
    text = re.sub("(Berbagai promo ini masih ditambah .+)$", "", text)
    text = re.sub("(Program ini untuk melengkapi promo .+)$", "", text)
    text = re.sub("(bagi yang bertransaksi .+)$", "", text)
    text = re.sub("(Untuk transaksi pembelian emas batangan .+)$", "", text)
    text = re.sub("(Untuk transaksi pembelian Emas .+)$", "", text)
    text = re.sub("(Dapatkan berbagai promo .+)$", "", text)
    text = re.sub("(Baca berita lainnya .+)$", "", text)
    
    return text.strip()
    
def scraping_one_article_detik(data_judul):
    """Scraping satu halaman tertentu dengan menggunakan
    data title yang sudah di scraping sebelumnya
    """
    
    iterasi = 0
    status_error = []
    data_per_tanggal = []

    for tanggal, df, in data_judul.groupby("date"):
        print(f"Scraping content article in detik finance website on {tanggal}")

        for index, row in df.iterrows():
            tmp = {'title' : row['title'], 'link' : row['link'], 'date' : tanggal}            

            try:
                URL = f"{tmp['link']}?single=1"
                page = requests.get(URL, headers={'user-agent': 'My app'})
                page = BeautifulSoup(page.content, 'html.parser')
                body_content = page.select(".detail__body-text")[0]
                
                # Hapus tags tertentu yang tidak mengandung makna
                remove_tags = ["div", "table", "style", "script", "strong"]
                for tag in remove_tags:
                    for html_tag in body_content.find_all(tag):
                        html_tag.decompose()
                
                # Hapus komentar html
                for child in body_content.children:
                    if isinstance(child,Comment):
                        child.extract()
                        
                paragraphs = body_content.find_all("p")
                if len(paragraphs) == 0:
                    raise Exception("no paragraph")
                
                # setiap paragraf akan dicek apakah ada isinya atau tidak
                # jika ada maka diakhir akan ditambahi titik dan spasi
                isi_artikel = []
                for paragraph in paragraphs:
                    text = paragraph.text.strip().rstrip('.')
                    if len(text) > 0:
                        text = text + '. ' 
                        isi_artikel.append(text)

                isi_artikel = " ".join(isi_artikel).strip()
                # jika artikelnya ternyata kosong, throw exception
                if len(isi_artikel) == 0:
                    raise Exception("no content")

                tmp['content'] = clean_artikel_detik(isi_artikel)
            except Exception as error:
                status_error.append({"date" : tanggal, "website" : "detik finance", 
                                     'link': tmp['link'], 'message' : str(error)})
                time.sleep(3)

            data_per_tanggal.append(tmp)
            iterasi += 1

            # setiap iterasi ke 200 istirahat 15 detik
            if iterasi % 200 == 0:
                time.sleep(10)
    
    # jika status errornya kosong maka buat dataframe kosong
    column_names = ["date", "website", "link", "message"]
    status_error = pd.DataFrame(status_error) if len(status_error) > 0 else pd.DataFrame(columns = column_names)
    return status_error, pd.DataFrame(data_per_tanggal)

## Another Function

In [7]:
def scraping_bisnis(article_related_to_saham):
    """Fungsi ini akan melakukan scraping pada 
    website bisnis.com kategori market dan financial
    dengan mengambil data dari tanggal terakhir data yang ada
    pada data article_related_to_saham.json sampai dengan kemarin
    """
    
    ordered_columns = ["title", "content", "category", "published_at", "link", "website"]
    bisnis_last_scraping, yesterday = last_scraping_website("bisnis.com", article_related_to_saham)

    # c = 194 adalah kategori artikel market di bisnis.com
    # c = 5 adalah kategori artikel financial di bisnis.com
    bisnis_market_status, bisnis_market_title = bisnis_scrape_title(bisnis_last_scraping, yesterday, category=194)
    bisnis_financial_status, bisnis_financial_title = bisnis_scrape_title(bisnis_last_scraping, yesterday, category=5)

    # Satukan data status scraping antara kategori market dan financial
    bisnis_status_scraping_title = pd.DataFrame(bisnis_market_status + bisnis_financial_status)

    # Satukan data title antara kategori market dan financial
    bisnis_article_title = (pd.concat([bisnis_market_title, bisnis_financial_title])
                            .sort_values("published_at")
                            .reset_index(drop=True))

    # Scraping full artikel
    bisnis_stop_words = pd.read_excel("data/stop_words_bisnis.xlsx").text.to_list()
    bisnis_status_error, bisnis_article = scraping_one_article_bisnis(bisnis_article_title, bisnis_stop_words)
    bisnis_article['website'] = "bisnis.com"

    # Ambil data category dengan published at pada tabel title
    bisnis_article = (bisnis_article
                      .set_index('link')
                      .join(bisnis_article_title
                            .set_index('link')[['category', 'published_at']])
                      .reset_index()[ordered_columns])
    
    print("\n")
    return bisnis_article, bisnis_status_scraping_title, bisnis_status_error

In [8]:
def scraping_detik(article_related_to_saham):
    """Fungsi ini akan melakukan scraping pada 
    website detik finance dengan mengambil data dari 
    tanggal terakhir data yang ada pada data 
    article_related_to_saham.json sampai dengan kemarin
    """
    
    ordered_columns = ["title", "content", "category", "published_at", "link", "website"]
    detik_last_scraping, yesterday = last_scraping_website("detik finance", article_related_to_saham)

    # scraping title
    detik_status_scraping_title, detik_article_title = detik_scrape_title(detik_last_scraping, yesterday)

    # scraping content
    detik_status_error, detik_article = scraping_one_article_detik(detik_article_title)
    detik_article['website'] = "detik finance"

    detik_article = (detik_article
                     .set_index('link')
                     .join(detik_article_title
                           .set_index('link')[['category', 'published_at']])
                     .reset_index()[ordered_columns])
    
    print("\n")
    return detik_article, detik_status_scraping_title, detik_status_error

In [9]:
def find_artikel_saham(row, all_saham):
    """Fungsi ini adalah fungsi callback
    yang digunakan untuk melihat apakah dalam satu artikel
    ada disebutkan nama saham atau nama perusahaan pemilik saham
    sehingga bisa diketahui artikel mana yang membahas saham
    dan mana yang tidak
    """
    title = row['title']
    content = str(row['content'])
    artikel_saham = []
    
    if content is not None:
        content_lower = content.lower()

        def contains_word(sentance, word):
            return (' ' + word + ' ') in (' ' + sentance + ' ')

        # untuk satu artikel akan dilakukan pengecekan untuk seluruh saham
        for index, saham in all_saham.iterrows():
            get_artikel = False
            
            kode_saham = saham['code']
            nama_perusahaan = saham['company_name']

            tmp = {"code" : kode_saham, "company_name" : nama_perusahaan, "link" : row['link']}

            # cek apakah content artikel mengandung kode saham
            # biasanya kode saham akan ditaruh di dalam kurung
            if contains_word(content, f"({kode_saham})"):
                tmp['keyword'] = kode_saham
                artikel_saham.append(tmp)
            # cek apakah artikel mengandung nama perusahaan tertentu
            elif contains_word(content, nama_perusahaan):
                tmp['keyword'] = nama_perusahaan
                artikel_saham.append(tmp)
            else:
                # jika kedua kondisi sebelumnya tidak terpenuhi maka akan dicek
                # apakah content yang telah dilower case mengandung keyword
                # yang telah ditentukan sebelumnya
                keywords = [] if saham['keyword'] is None else str(saham['keyword']).split(';')

                for keyword in keywords:
                    if contains_word(content_lower, keyword):
                        tmp['keyword'] = keyword
                        artikel_saham.append(tmp)
                        break                        
    
    # jika tidak ditemukan maka isikan kolom baru dengan None bukan 
    # dengan array kosong sehingga nanti mudah untuk diidentifikasi menggunakan fungsi isnull
    row['related_to_saham'] = artikel_saham if len(artikel_saham) > 0 else None
    return row

In [10]:
def backup_data():
    """Fungsi ini akan melakukan backup data
    kedalam folder backup. Setiap file hasil backup
    akan ditambahkan prefix timestamp
    """
    if not os.path.exists('data/backup'):
        os.makedirs('data/backup')

    print("Melakukan Backup Data")
    now = int(datetime.datetime.now().timestamp())
    shutil.copyfile("data/article_related_to_saham.json", f"data/backup/{now}_article_related_to_saham.json")
    shutil.copyfile("data/article_norelated_to_saham.json", f"data/backup/{now}_article_norelated_to_saham.json")
    shutil.copyfile("data/table_articles.json", f"data/backup/{now}_table_articles.json")
    shutil.copyfile("data/table_article_saham.json", f"data/backup/{now}_table_article_saham.json")
    
    if os.path.exists("data/status_scraping_title.xlsx"):
        shutil.copyfile("data/status_scraping_title.xlsx", f"data/backup/{now}_status_scraping_title.xlsx")
        
    if os.path.exists("data/status_error.xlsx"):
        shutil.copyfile("data/status_error.xlsx", f"data/backup/{now}_status_error.xlsx")

In [11]:
def update_status_scraping_title(status_scraping_title):
    """Fungsi ini akan melakukan update file
    excel status_scraping_title yang menyimpan informasi
    apakah scraping title pada tanggal tertentu gagal atau tidak
    """
    
    # cek apakah filenya sudah ada atau belum. 
    # Jika ada read file tersebut akan tetapi 
    # jika tidak maka buat data frame kosong yang baru
    if os.path.exists("data/status_scraping_title.xlsx"):
        status_scraping_title_old = pd.read_excel("data/status_scraping_title.xlsx")
    else:
        status_scraping_title_old = pd.DataFrame(columns=["date", "status", "article_count", "website", "message"])

    (pd.concat([status_scraping_title_old, status_scraping_title])
     .reset_index(drop=True).to_excel("data/status_scraping_title.xlsx", index=False))

In [12]:
def update_status_error(status_error):
    """Fungsi ini akan melalukan update file
    status_error yang mana merupakan file yang akan 
    menyimpan data link yang gagal dilakukan scraping beserta alasannya
    """
    
    # cek apakah filenya sudah ada atau belum. 
    # Jika ada read file tersebut akan tetapi 
    # jika tidak maka buat data frame kosong yang baru
    if os.path.exists("data/status_error.xlsx"):
        status_error_old = pd.read_excel("data/status_error.xlsx")
    else:
        status_error_old = pd.DataFrame(columns=["date", "website", "link", "message"])

    (pd.concat([status_error_old, status_error])
     .reset_index(drop=True).to_excel("data/status_error.xlsx", index=False))

In [13]:
def update_data_saham(start="01-01-2000 08"):
    """Fungsi ini untuk melakukan update
    pada data saham
    """
    start = datetime.datetime.strptime(start, "%d-%m-%Y %H")
    start = int(datetime.datetime.timestamp(start))
    
    end = datetime.datetime.now()
    end = end.strftime("%d-%m-%Y") + " 08"
    end = datetime.datetime.strptime(end, "%d-%m-%Y %H") - datetime.timedelta(days=1)
    end = int(datetime.datetime.timestamp(end))
    
    if not os.path.exists('data/harga_saham'):
        os.makedirs('data/harga_saham')
    
    list_saham = ["BBNI", "BBRI", "BMRI", "BBCA"]  
    for saham in list_saham:
        link = f"https://query1.finance.yahoo.com/v7/finance/download/{saham}.JK?period1={start}&period2={end}&interval=1d&events=history&includeAdjustedClose=true"
        page = requests.get(link, headers={'user-agent': 'My app'})
        open(f'data/harga_saham/{saham}.csv', 'wb').write(page.content)
    print("Saham data has been updated!")

In [14]:
def check_data():
    """Fungsi ini digunakan untuk melakukan pengecekan sederhana
    setelah data berhasil diupdate. Hanya melakukan pengecekan
    ke konsistenan data
    """
    tb_articles = pd.read_json('data/table_articles.json')
    tb_articles_saham = pd.read_json('data/table_article_saham.json')
    has_error = False
    
    if tb_articles.id.max() != tb_articles.shape[0]:
        print("Id Article tidak cocok")
        has_error = True
        
    if tb_articles.link.duplicated().sum() != 0:
        print("Terdapat duplikasi pada tabel article")
        has_error = True
        
    if tb_articles_saham[['saham_id', 'article_id']].duplicated().sum() != 0:
        print("Terdapat duplikasi pada tabel article saham")
        has_error = True
    
    if not has_error:
        print("Update data aman. Mantaaap!!!")

## Main

In [15]:
article_related_to_saham = pd.read_json("data/article_related_to_saham.json")

bisnis_article, bisnis_status_scraping_title, bisnis_status_error = scraping_bisnis(article_related_to_saham)
detik_article, detik_status_scraping_title, detik_status_error = scraping_detik(article_related_to_saham)

# Gabung antara artikel di detik dan bisnis.com
article = pd.concat([detik_article, bisnis_article]).sort_values('published_at').reset_index(drop=True)

# Gabung status scraping title
status_scraping_title = (pd.concat([detik_status_scraping_title, bisnis_status_scraping_title])
                         .sort_values(['date', 'website'])
                         .reset_index(drop=True))

# Gabung status error scraping artikel
status_error = (pd.concat([detik_status_error, bisnis_status_error])
                .sort_values(['date', 'website'])
                .reset_index(drop=True))

# Indentifikasi artikel mana saja yang berkaitan dengan saham atau perusahaan pemilik saham
all_saham = pd.read_csv("data/daftar_saham.csv")
article = article.apply(lambda row : find_artikel_saham(row, all_saham), axis=1)

# Pisahkan artikel yang tidak berkaitan dengan saham
article_norelated_to_saham_new = (article[article.related_to_saham.isnull()]
                       .drop('related_to_saham', axis=1)
                       .reset_index(drop=True))

# Pisahkan artikel yang berkaitan dengan saham
article_related_to_saham_new = (article[article.related_to_saham.notnull()]
                       .reset_index(drop=True))

# ambil informasi yang ada di kolom related to saham
related_to_saham = []
for index, row in article_related_to_saham_new.iterrows():
    related_to_saham = related_to_saham + row['related_to_saham']
related_to_saham = pd.DataFrame(related_to_saham)

# ambil hanya artikel yang berkaitan dengan bank yang telah ditentukan
# yang ada pada data table_saham.json
table_saham = pd.read_json('data/table_saham.json')
selected_bank = table_saham.company_name.to_list()
selected_artikel_bank = related_to_saham[related_to_saham.company_name.isin(selected_bank)]

# ambil nilai id maksimal dari artikel yang sekarang sehingga 
# nilai id akan terus berlanjut
table_article = pd.read_json("data/table_articles.json")
max_id = table_article.id.max()

# artikel baru yang akan dimasukkan ke tabel article
table_article_new = (article_related_to_saham_new
                     .loc[article_related_to_saham_new.link.isin(selected_artikel_bank.link.to_list()), :]
                     .reset_index(drop=True)
                     .reset_index()
                     .rename(columns={"index" : "id"}))
table_article_new['id'] = table_article_new['id'] + 1 + max_id

# article saham baru yang akan dimasukkan ke tabel article saham
table_article_saham_new = (selected_artikel_bank
                         .set_index("code")
                         .join(table_saham[['id', "code"]]
                               .set_index('code'))
                         .rename(columns={"id" : "saham_id"})
                         .set_index("link")
                         .join(table_article_new[["id", "link"]]
                               .set_index("link"))
                         .rename(columns={"id" : "article_id"})
                         .drop(["company_name"], axis=1)
                         .reset_index(drop=True))

# Gabung data antara data yang baru dengan 
# data yang lama yang sudah disimpan sebelumnya
table_article = pd.concat([table_article, table_article_new]).reset_index(drop=True)

table_article_saham = pd.read_json("data/table_article_saham.json")
table_article_saham = pd.concat([table_article_saham, table_article_saham_new]).reset_index(drop=True)

article_related_to_saham = pd.concat([article_related_to_saham, article_related_to_saham_new]).reset_index(drop=True)

article_norelated_to_saham = pd.read_json("data/article_norelated_to_saham.json")
article_norelated_to_saham = (pd.concat([article_norelated_to_saham, article_norelated_to_saham_new])
                                  .reset_index(drop=True))

# update data ke file, sebelum dilakukan update akan dilakukan
# backup data terlebih dahulu ke folder backup
# sehingga jika ada kesalahan datanya tidak akan hilang
backup_data()
table_article_saham.to_json('data/table_article_saham.json', orient="records")
table_article.to_json('data/table_articles.json', orient="records")
article_related_to_saham.to_json("data/article_related_to_saham.json", orient="records")
article_norelated_to_saham.to_json("data/article_norelated_to_saham.json", orient="records")
update_status_scraping_title(status_scraping_title)
update_status_error(status_error)
print("Data has been updated!")
update_data_saham()
check_data()

Start Scraping bisnis.com, Market from 2021-12-31 until 2021-12-31
Start Scraping title in bisnis.com, Market category on 2021-12-31
Scraping 2021-12-31 success, numbers of article is 29

Start Scraping bisnis.com, Financial from 2021-12-31 until 2021-12-31
Start Scraping title in bisnis.com, Financial category on 2021-12-31
Scraping 2021-12-31 success, numbers of article is 29

Scraping content article of bisnis.com on 2021-12-31


Start Scraping detik finance from 2021-12-31 until 2021-12-31
Start Scraping title in detik finance website on 2021-12-31
Scraping 2021-12-31 success, numbers of article is 96

Scraping content article in detik finance website on 2021-12-31


Melakukan Backup Data
Data has been updated!
Saham data has been updated!
Update data aman. Mantaaap!!!


In [16]:
tb_article = pd.read_json("data/table_articles.json")
tb_article = tb_article[tb_article.published_at.notnull()]
tb_article.loc[:, 'date'] = tb_article.published_at.dt.strftime("%Y-%m-%d")

for date, df in tb_article.groupby("date"):
    df = df.drop("date", axis=1)
    df.to_json(f"data/article/{date}.json", orient="records")